---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [15]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [20]:
def answer_one():
    df = pd.read_table('Employee_Movie_Choices.txt')
    G = nx.from_pandas_dataframe(df, '#Employee', 'Movie')
    return G

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [21]:
def answer_two():
    G_2 = answer_one()
    G_2.add_nodes_from(employees, type='employee')
    G_2.add_nodes_from(movies, type='movie')
    return G_2

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [22]:
def answer_three():
    G_2 = answer_two()
    G_2_weighted = nx.bipartite.weighted_projected_graph(G_2, employees)
    return G_2_weighted

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [23]:
def answer_four():
    relationships = pd.read_table('Employee_Relationships.txt',
                                  header=None)
    relationships.rename(columns={0:'from',
                                  1:'to',
                                  2:'relationships'}, inplace=True)
    common_movies = pd.DataFrame.from_dict(answer_three().edges(data=True))
    common_movies[2] = common_movies[2].apply(lambda x: x['weight'])
    
    for index, row in relationships.iterrows():
        e1a = row[0]
        e2a = row[1]
        for index_2, row_2 in common_movies.iterrows():
            e1b = row_2[0]
            e2b = row_2[1]
            if ((e1a == e1b) & (e2a == e2b)) | ((e1a == e2b) & (e2a == e1b)):
                relationships.loc[index, 'common_movies'] = row_2[2]
    relationships['common_movies'].fillna(0, inplace=True)
    corr = relationships.corr()['common_movies'][0]
    return corr

In [24]:
answer_four()

0.78839622217334759